In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cracks-detection/val/good/00460.jpg
/kaggle/input/cracks-detection/val/good/00439.jpg
/kaggle/input/cracks-detection/val/good/00428.jpg
/kaggle/input/cracks-detection/val/good/00468.jpg
/kaggle/input/cracks-detection/val/good/00386.jpg
/kaggle/input/cracks-detection/val/good/00498.jpg
/kaggle/input/cracks-detection/val/good/00516.jpg
/kaggle/input/cracks-detection/val/good/00413.jpg
/kaggle/input/cracks-detection/val/good/00381.jpg
/kaggle/input/cracks-detection/val/good/00411.jpg
/kaggle/input/cracks-detection/val/good/00387.jpg
/kaggle/input/cracks-detection/val/good/00380.jpg
/kaggle/input/cracks-detection/val/good/00511.jpg
/kaggle/input/cracks-detection/val/good/00485.jpg
/kaggle/input/cracks-detection/val/good/00465.jpg
/kaggle/input/cracks-detection/val/good/00512.jpg
/kaggle/input/cracks-detection/val/good/00397.jpg
/kaggle/input/cracks-detection/val/good/00496.jpg
/kaggle/input/cracks-detection/val/good/00442.jpg
/kaggle/input/cracks-detection/val/good/00475.jpg


In [8]:
from tensorflow.keras.layers import Input,Dense,Flatten,Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet_v2 import ResNet152V2 
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [9]:
IMAGE_SIZE = [224,224]

In [10]:
train_path = r'/kaggle/input/cracks-detection/train'
valid_path = r'/kaggle/input/cracks-detection/test'

In [11]:
resnet152v2 =ResNet152V2(input_shape=IMAGE_SIZE + [3], weights='imagenet',include_top=False)

2023-02-05 10:01:29.862317: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-05 10:01:29.862367: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-05 10:01:29.862393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (4afe7c1ba7e7): /proc/driver/nvidia/version does not exist
2023-02-05 10:01:29.862699: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


234545216/234545216 [==============================] - 1s 0us/step


In [12]:
for layer in resnet152v2.layers:    #layer.trainable=False means we dont want to retrain the existing weights.
    layer.trainable=False

In [13]:
folders=glob(r'/kaggle/input/cracks-detection/train/*')
folders

['/kaggle/input/cracks-detection/train/good',
 '/kaggle/input/cracks-detection/train/crack']

In [14]:
resnet152v2.output

<KerasTensor: shape=(None, 7, 7, 2048) dtype=float32 (created by layer 'post_relu')>

In [15]:
x= Flatten()(resnet152v2.output)

In [16]:
prediction = Dense(len(folders),activation='softmax')(x)

In [18]:
model = Model(inputs=resnet152v2.input, outputs=prediction)

In [19]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range = 0.2, zoom_range=0.2, horizontal_flip=True)

In [22]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [23]:
training_set=train_datagen.flow_from_directory(r'/kaggle/input/cracks-detection/train',target_size=(224,224),batch_size=32,class_mode='categorical')

Found 37709 images belonging to 2 classes.


In [24]:
test_set=test_datagen.flow_from_directory(r'/kaggle/input/cracks-detection/test',target_size=(224,224),batch_size=32,class_mode='categorical')

Found 13 images belonging to 2 classes.


In [25]:
r=model.fit(training_set, validation_data=test_set,epochs=5,steps_per_epoch=len(training_set),validation_steps=len(test_set))

Epoch 1/5
1179/1179 [==============================] - 1104s 927ms/step - loss: 0.0798 - accuracy: 0.9934 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/5
1179/1179 [==============================] - 1076s 913ms/step - loss: 0.0646 - accuracy: 0.9962 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/5
1179/1179 [==============================] - 1074s 910ms/step - loss: 0.0696 - accuracy: 0.9970 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/5
1179/1179 [==============================] - 1059s 898ms/step - loss: 0.0657 - accuracy: 0.9972 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/5
1179/1179 [==============================] - 1055s 895ms/step - loss: 0.0415 - accuracy: 0.9982 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [6]:
import matplotlib.pyplot as plt

In [7]:

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

NameError: name 'r' is not defined